In [0]:
from pyspark.sql.functions import *
import urllib

## Defining the data cleaning methods:

The following transformations are required to process the data effectively:
#### Pin data:
- Replace missing or non-applicable values with None
- Adjust the numerical series so they only contain numbers

  _e.g. in follower_count '100k' should read '100000'_
- Update data types where they are inaccurate
- Remove the unnecessary "Local save in" prefix from values in the save_location series
- Rename 'index' series to match other two dataframes

In [0]:
def clean_pin_df(df):
  """
  Cleans the pin dataframe by performing the following transformations:
  - Replaces the values in the description series with None if they match the values in the to_replace_with_none dictionary
  - Converts k, M and B suffixes with 000, 000000 and 000000000 respectively in the follower_count series
  - Converts all data types to numeric where applicable
  - Removes the unnecessary "Local save in " prefix from the save_location series
  - Renames the index series to 'ind' to match geo and user dataframes
  - Restructures columns in more logical order
  """
  to_replace_with_none = {
    'description': ['No description available Story format', 'Untitled', 'No description available Story format'],
    'image_src': 'Image src error.',
    'poster_name': 'User Info Error',
    'tag_list': 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e',
    'title': 'No Title Data Available'
    }

  for column, values in to_replace_with_none.items():
    if isinstance(values, list):
        for value in values:
          cleaned_df = df.replace(value, None, subset=[column])
    else:
        cleaned_df = cleaned_df.replace(values, None, subset=[column])                    

  cleaned_df = cleaned_df.replace({'User Info Error':'0'}, subset=['follower_count'])

  cleaned_df = cleaned_df.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
  cleaned_df = cleaned_df.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
  cleaned_df = cleaned_df.withColumn("follower_count", regexp_replace("follower_count", "B", "000000000"))

  cleaned_df = cleaned_df.withColumn("follower_count", cleaned_df["follower_count"].cast("int"))

  cleaned_df = cleaned_df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

  cleaned_df = cleaned_df.withColumnRenamed("index", "ind")

  cleaned_df = cleaned_df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")
  return cleaned_df
  

#### Geo data:
- Create a series named 'coordinates' by joining the both the 'latitude' and 'longitude' columns, seperating the values with a comma
- Convert the 'timestamp' column to timestamp type

In [0]:
def clean_geo_df(df):
    """
  Cleans the geo dataframe by performing the following transformations:
  - New series 'coordinates' created from latitude and longitude series
  - Timestamp series converted to timestamp type
  - Restructures columns in more logical order
  """
  cleaned_df = df.withColumn("coordinates", array("latitude", "longitude"))
  cleaned_df = cleaned_df.drop("latitude", "longitude")
  cleaned_df = cleaned_df.withColumn("timestamp", to_timestamp("timestamp"))
  cleaned_df = cleaned_df.select("ind", "country", "coordinates", "timestamp")
  return cleaned_df

#### User data
- Create a series 'user_name' by joining the 'first_name' and 'last_name' series and then dropping them
- Convert 'date_joined' series to timestamp type

In [0]:
def clean_user_df(df):
    """
    Cleans the geo dataframe by performing the following transformations:
    - Creates new series 'user_name' created from 'first_name' and 'last_name'
    -  Drops 'first_name' and 'last_name' series
    - 'date_joined' series converted to timestamp type
    - Restructures columns in more logical order
    """
  cleaned_df = df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
  cleaned_df = cleaned_df.drop("first_name", "last_name")
  cleaned_df = cleaned_df.withColumn("date_joined", to_timestamp("date_joined"))
  cleaned_df = cleaned_df.select("ind", "user_name", "age", "date_joined")
  return cleaned_df

In [0]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.pin/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
cleaned_pin_df = clean_pin_df(pin_df)

display(cleaned_pin_df)

In [0]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.geo/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

cleaned_geo_df = clean_geo_df(geo_df)

display(cleaned_geo_df)

In [0]:
geo_df.dtypes

#### Load the data from S3 into Databricks and apply relevant cleaning methods:

In [0]:
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.user/partition=0/*.json" 
file_type = "json" 
infer_schema = "true"
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

cleaned_user_df = clean_user_df(user_df)

display(cleaned_user_df)

#### Query to find the most popular post category by country
This is done by joining the geo_df with the pin_df(using 'ind' as the common series), then grouping by both the category and country columns and applying the `count` aggregation function:

In [0]:
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").select("country", "category")

popular_category_df = joined_df.groupBy("country", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").orderBy("category_count", ascending=False)

display(popular_category_df)

#### Query to find the most popular post category by year
To do this, the 'post_year' is first extracted from the timestamp into a new series. The dataframe is then grouped with respect to this series and the count aggregator applied like before.
##### Sub query: How many posts did each category have between 2018 and 2022?
With the 'post_year' series created, this is simply a matter of grouping the dataframe by year, then category and sorting the resulting series from highest to lowest. The result must also be filtered to only include records from 2018-2022.

In [0]:
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").withColumn("post_year", year("timestamp")).select("timestamp", "category")

popular_category_by_year_df = joined_df.withColumn("post_year", year("timestamp")).select("post_year", "category").groupBy("post_year", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").filter((col("post_year") >= 2018) & (col("post_year") <= 2022)).orderBy("category_count", ascending=False)

display(popular_category_by_year_df)

#### Query to find the user with the most followers in each country
This is a matter of joining the 'geo' and 'pin' dataframes and sorting them by 'follower_count' in descending order.

This information can then be used to easily identify the user with the most followers.

In [0]:
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").select("country", "poster_name", "follower_count").orderBy("follower_count", ascending=False)

display(joined_df)

In [0]:
highest_user = joined_df.agg(max("follower_count").alias("highest_user")).collect()[0]["highest_user"]

result = joined_df.filter(col("follower_count") == highest_user).select("country", "follower_count")

display(result)

#### Query to find the most popular category for different age groups
This is achieved by joining the 'user' and 'pin' dataframes, sorting them by some common age groups, counting the number of records per group for each category, and then sorting the results from highest to lowest.

In [0]:
joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "category")

temp_df = joined_df.withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
)

result = temp_df.select("age_group", "category").groupBy("age_group", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").orderBy("category_count", ascending=False)


display(result)

#### Query to find the median follower count over different age groups
Thankfully, spark has a function for calculating median available in the `functions` library.
The syntax is as follows:

`
df.agg(functions.expr("percentile_approx(follower_count, 0.5)"))
`

Where the `0.5` refers to the 50th percentile.

The dataframe is then grouped by the common age ranges and a new series is created to display the median follower count by age group.


In [0]:
from pyspark.sql import functions as F

joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "follower_count").withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
)
temp_df = joined_df.select("age_group", "follower_count")
agg_df = joined_df.groupBy("age_group") \
    .agg(
        F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")
    ).orderBy("median_follower_count", ascending=False)

display(agg_df)

#### Query to check how many users joined in each year between 2015 & 2020
Achieved by joining the geo and user dataframes, extracting the year from the 'date_joined' series and counting the users by year. Finally the result is filtered to only include records where the 'join_year' is between 2015 and 2020.

In [0]:

joined_df = cleaned_user_df.join(cleaned_geo_df, cleaned_user_df.ind == cleaned_geo_df.ind, "inner").select("timestamp", "date_joined").withColumn("post_year", year("timestamp")).withColumn("join_year",year("date_joined"))

grouped_df = joined_df.groupBy("join_year").agg(count("join_year").alias("number_users_joined")).filter((col("join_year") >= 2015) & (col("join_year") <= 2020)).orderBy("join_year", ascending=True)
display(grouped_df)

#### Query to find the median follower count of users that have joined between 2015 and 2020

Again, this makes use of `functions.expr("percentile_approx(follower_count, 0.5)")`, creating a dataframe of median follower counts grouped by join year, once again filtered to exclude records before 2015 or after 2020.

In [0]:
joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("date_joined","follower_count").withColumn("join_year",year("date_joined"))
temp_df = joined_df.select("join_year", "follower_count")
result = temp_df.groupBy("join_year").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")).filter((col("join_year") >= 2015) & (col("join_year") <= 2020)).orderBy("join_year", ascending=True)

display(result)

#### Query to find the median follower count of users based on their joining year and age group
Similar to the previous query, except now the "age_group" series has been reintroduced. By grouping by age range and _then_ by join year, we are able to split the data in the previous dataframe and look at more specific insights.

In [0]:
joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "follower_count", "date_joined")

temp_df = joined_df.withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
).withColumn("join_year",year("date_joined")).select("age_group", "join_year", "follower_count").filter("join_year >= 2015 and join_year <= 2020")
result = temp_df.groupBy("age_group", "join_year").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
display(result)